In [3]:
pip install gym

     |████████████████████████████████| 1.6 MB 134 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 366 kB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657513 sha256=bd5bd7a9ecbbd54e5d7691a38d88ad5e4b3d9c1f87c7c245d2a2e3d7c03e280d
  Stored in directory: /Users/admin/Library/Caches/pip/wheels/b3/03/54/9141c232861b89be935b37bdde0ea5ab472f5e18fc20623aed
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gym
import numpy as np
import random

In [6]:
e = gym.make("Taxi-v3").env
e.render()

e.reset()
#show image
e.render()


#6 spaces as it takes pick and drop + 4 directions
print (e.action_space)

#total states
print (e.observation_space)

e.s = e.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
e.render()

print (e.s)
print (e.P[328])

epochs, penalties, reward = 0, 0, 0
while not reward == 20:
    action = e.action_space.sample()
    state, reward, done, info = e.step(action) # (284, -1, False, {'prob': 1.0})
    if reward == -10:
        penalties += 1     
    epochs += 1      
print("Timesteps taken:",epochs)
print("Penalties incurred:",penalties)

q_table = np.zeros([e.observation_space.n, e.action_space.n])
print (q_table[328])
total_epochs, total_penalties = 0, 0
for i in range(1, 100):
    state = e.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False        
    while not done:
        if random.uniform(0, 1) < 0.1:
            action = e.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = e.step(action)          
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])       
        new_value = (1 - 0.1) * old_value + 0.1 * (reward + 0.6 * next_max)
        q_table[state, action] = new_value
        if reward == -10:
            penalties += 1
        state = next_state
        epochs += 1 
    total_penalties += penalties
    total_epochs += epochs              

print (q_table[328])  
print("Timesteps taken:",epochs)
print("Penalties incurred:",penalties)
print("Average timesteps per episode:",(total_epochs / 100))
print("Average penalties per episode",(total_penalties / 100))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Discrete(6)
Discrete(500)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

328
{0: [(1.0, 428, -1, False)], 1: [(1.0, 228, -1, False)], 2: [(1.0, 348, -1, False)], 3: [(1.0, 328, -1, False)], 4: [(1.0, 328, -10, False)], 5: [(1.0, 328, -10, False)]}
Timesteps taken: 65
Penalties incurred: 29
[0. 0. 0. 0. 0. 0.]
[-1.4724725  -1.50811951 -1.48930394 -1.48162639 -3.62609543 -2.79359326]
Timesteps taken: 299
Penalties incurred: 15
Average timesteps per episode: 456.05
Average penalties per episode 24.76
